In [1]:

import sentier_data_tools as sdt
import os 

import brightway2 as bw
import lca_algebraic as agb
import numpy as np
from sympy import init_printing
import bw2io
import bw2data as bd
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.max_rows', 500)
# Pretty print for Sympy
init_printing()

In [ ]:
ECOINVENT_LOGIN="YOUR_LOGIN" #suppress login before any git push
ECOINVENT_PASSWORD="YOUR_PASSWORD"   #suppress password before any git push

In [3]:
bw.projects.set_current('PCB_Project')


# This load .env file into os.environ
load_dotenv()

# This downloads ecoinvent and installs biopshere + technosphere + LCIA methods
if len(bw.databases) > 0:
    print("Initial setup already done, skipping")
   
else:
    # This is now the prefered method to init an Brightway2 with Ecoinvent
    # It is not more tied to a specific version of bw2io
    bw2io.import_ecoinvent_release(
        version="3.10",
        system_model="cutoff",
        username=ECOINVENT_LOGIN, # Read for .env file
        password=ECOINVENT_PASSWORD, # Read from .env file
        use_mp=True)
bw.databases

Initial setup already done, skipping


Databases dictionary with 2 object(s):
	ecoinvent-3.10-biosphere
	ecoinvent-3.10-cutoff

In [78]:
EcoInvent = "ecoinvent-3.10-cutoff"
USER_DB = 'MyForeground'
#agb.setForeground(DB_Buck)
# This is better to cleanup the whole foreground model each time, and redefine it in the notebook (or a python file)
# instead of relying on a state or previous run.
# Any persistent state is prone to errors.
agb.resetDb(USER_DB)
# Parameters are stored at project level : 
# Reset them also
# You may remove this line if you import a project and parameters from an external source (see loadParam(..))
agb.resetParams()

# Overview of the databases
agb.list_databases()

,backend,nb_activities,type
name,,,
ecoinvent-3.10-biosphere,sqlite,4363,biosphere
ecoinvent-3.10-cutoff,sqlite,23523,background
MyForeground,sqlite,0,foreground


In [83]:
essai = pd.read_excel('lci-MyForeground.xlsx')
essai

,Database,MyForeground,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17
0,fg,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Activity,"printed wiring board production, for surface m...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,activity,21483848-7260-50c0-8799-0f413c2ca2d8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,activity type,ordinary transforming activity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,code,"printed wiring board production, for surface m...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,comment,The dataset represents the assembly of 1 m2 o...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,filename,21483848-7260-50c0-8799-0f413c2ca2d8_65cf8186-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,flow,65cf8186-2a0d-4271-b7ee-62c9f5d5100c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,inherited_from,ecoinvent-3.10-cutoff::462afea645c48b61113c6ec...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
df_complete = [['AOX, Adsorbable Organic Halides', 3.624321e-05, 'kilogram'],
       ['Arsenic ion', 5.911781e-07, 'kilogram'],
       ['BOD5, Biological Oxygen Demand', 0.02218105, 'kilogram'],
       ['COD, Chemical Oxygen Demand', 0.02218105, 'kilogram'],
       ['Cadmium II', 5.0005e-6, 'kilogram'],
       ['Chromium III', 1.416e-5, 'kilogram'],
       ['Copper ion', 0.00498, 'kilogram'],
       ['Copper ion', 4.763029e-05, 'kilogram'],
       ['DOC, Dissolved Organic Carbon', 0.0082152037037037, 'kilogram'],
       ['Fluoride', 0.0001019924, 'kilogram'],
       ['Hydrochloric acid', 0.00077535, 'kilogram'],
       ['Hydrogen peroxide', 9.3578e-5, 'kilogram'],
       ['Mercury II', 1.6537e-7, 'kilogram'],
       ['NMVOC, non-methane volatile organic compounds', 0.01158578, 'kilogram'],
       ['Nickel II', 1.6537e-5, 'kilogram'],
       ['Particulate Matter, < 2.5 um', 9.134908e-05, 'kilogram'],
       ['Particulate Matter, > 2.5 um and < 10um', 3.04506e-05, 'kilogram'],
       ['Sodium hydroxide', 0.00040103, 'kilogram'],
       ['Sulfuric acid', 6.684e-5, 'kilogram'],
       ['TOC, Total Organic Carbon', 0.0082152037037037, 'kilogram'],
       ['Water', 0.11081, 'cubic meter'],
       ['Water', 1.7058, 'cubic meter'],
       ['Water, unspecified natural origin',1.515 , 'cubic meter'],
       ['Zinc II', 1.3952e-5, 'kilogram'],
       ['market for blast furnace sludge[IN]', -0.6883736601588775, 'kilogram'],
       ['market for blast furnace sludge[Europe without Switzerland]', -0.1978571170468231, 'kilogram'],
       ['market for blast furnace sludge[CH]', -0.007894408540622268,'kilogram'],
       ['market for blast furnace sludge[RoW]', -0.22482181425367695,'kilogram'],
       ['market for copper, cathode', 0.7802932, 'kilogram'],
       ['market for dipropylene glycol monomethyl ether[RER]', 0.02364821820000009, 'kilogram'],
       ['market for dipropylene glycol monomethyl ether[RoW]', 0.11226108179999991, 'kilogram'],
       ['market group for electricity, medium voltage', 88.62649, 'kilowatt hour'],
       ['market for electronic component factory', 1e-12, 'unit'],
       ['market for glass fibre reinforced plastic, polyester resin, hand lay-up', 0.5129479, 'kilogram'],
       ['market for gold', 0.00015474, 'kilogram'],
       ['market for hazardous waste, for incineration[Europe without Switzerland]', -1.214841922, 'kilogram'],
       ['market for hazardous waste, for incineration[CH]', -0.0317552342564274, 'kilogram'],
       ['market for hazardous waste, for incineration[RoW]', -2.5262038437435725, 'kilogram'],
       ['market group for heat, district or industrial, natural gas', 102.9832, 'megajoule'],
       ['market for hydrochloric acid, without water, in 30% solution state[RoW]', 3.03849288685274, 'kilogram'],
       ['market for hydrochloric acid, without water, in 30% solution state[RER]',  0.6650901131472601, 'kilogram'],
       ['market for hydrogen peroxide, without water, in 50% solution state[RER]', 0.083566284, 'kilogram'],
       ['market for hydrogen peroxide, without water, in 50% solution state[RoW]', 0.396699716, 'kilogram'],
       ['market for iron(III) chloride, without water, in 40% solution state', 0.2336944, 'kilogram'],
       ['market for municipal solid waste[RoW]', -0.013593327589779514, 'kilogram'],
       ['market for municipal solid waste[IN]', -2.860953426935649e-06, 'kilogram'],
       ['market for municipal solid waste[KR]', -1.4524539933431455e-06, 'kilogram'],
       ['market for municipal solid waste[TR]', -6.426008576609077e-07,  'kilogram'],
       ['market group for municipal solid waste[RER]', -0.00046918164489892007, 'kilogram'],
       ['market for municipal solid waste[RU]', -1.8177681795476327e-05, 'kilogram'],
       ['market for municipal solid waste[TW]', -2.40931307865466e-05, 'kilogram'],
       ['market for municipal solid waste[JP]', -6.046609988044882e-05, 'kilogram'],
       ['market for municipal solid waste[CY]', -2.0019832120543873e-06, 'kilogram'],
       ['market for municipal solid waste[CA]', -1.780586136909808e-05,'kilogram'],
       ['market for nickel, class 1', 0.00077474, 'kilogram'],
       ['market for phenolic resin[RoW]', 0.17620232000112065, 'kilogram'],
       ['market for phenolic resin[RER]', 0.037117679998879355, 'kilogram'],
       ['market for sheet rolling, copper', 0.4931325, 'kilogram'],
       ['market for silver', 0.00051474, 'kilogram'],
       ['market for sodium chloride, powder', 0.1386319, 'kilogram'],
       ['market for sodium hydroxide, without water, in 50% solution state[RoW]', 1.7230844587761323, 'kilogram'],
       ['market for sodium hydroxide, without water, in 50% solution state[RER]', 0.16825654122386746, 'kilogram'],
       ['market for sulfuric acid[RER]', 0.036653177780449415, 'kilogram'],
       ['market for sulfuric acid[RoW]', 0.2802306222195506, 'kilogram'],
       ['market group for tap water', 6.5057, 'kilogram'],
       ['market for tin', 0.0027884, 'kilogram'],
       ['market for waste emulsion paint[RoW]', -0.04423000086520157,'kilogram'],
       ['market for waste emulsion paint[Europe without Switzerland]',-0.005179359705628613, 'kilogram'],
       ['market for waste emulsion paint[CH]', -4.319429169821787e-06,'kilogram'],
       ['market for water, ultrapure[RER]', 69.43301243189744, 'kilogram'],
       ['market for water, ultrapure[RoW]', 96.851700273201, 'kilogram'],
       ['market for water, ultrapure[CA-QC]', 128.8021872949016,'kilogram']
]

# 